In [5]:
from mqt.bench import *
from qiskit import *
from qiskit.circuit.library import *
from qiskit.quantum_info import *
from qiskit.algorithms import *
from qiskit import *
from qiskit.providers.fake_provider import FakeSherbrooke
from qiskit.circuit.random import random_circuit
import numpy as np
from quantumCircuits import buildQC

backend = FakeSherbrooke()


In [6]:
#Quantum Volume stays the same depth, considering  adding depth keyword 
def quantum_voulme(num_qubits):
    return QuantumVolume(num_qubits, seed = 42)

In [9]:
def grovers_alg(good_state, iterations):
    oracle = Statevector.from_label(good_state)
    problem = AmplificationProblem(oracle, is_good_state = good_state)
    grover = Grover(iterations=iterations)
    circuit = grover.construct_circuit(problem)
    circuit.measure_all()
        
    return circuit

In [10]:
def bernstein_vazarani(n,s):
    # We need a circuit with n qubits, plus one auxiliary qubit
    # Also need n classical bits to write the output to
    bv_circuit = QuantumCircuit(n+1, n)

    # put auxiliary in state |->
    bv_circuit.h(n)
    bv_circuit.z(n)

    # Apply Hadamard gates before querying the oracle
    for i in range(n):
        bv_circuit.h(i)
        
    # Apply barrier 
    bv_circuit.barrier()

    # Apply the inner-product oracle
    s = s[::-1] # reverse s to fit qiskit's qubit ordering
    for q in range(n):
        if s[q] == '0':
           continue  #bv_circuit.i(q)
        else:
            bv_circuit.cx(q, n)
            
    # Apply barrier 
    bv_circuit.barrier()

    #Apply Hadamard gates after querying the oracle
    for i in range(n):
        bv_circuit.h(i)

    # Measurement
    for i in range(n):
        bv_circuit.measure(i, i)

    return bv_circuit

In [11]:
def dj_algorithm_balnced(num_qubits):
    return None 

def dj_balanced(num_qubits):
    constant_oralce = QuantumCircuit(num_qubits+1)
    output = np.random.randint(2)
    if output == 1:
        constant_oralce.x(n)

def dj_constant():
    return None 

In [12]:
def error_correction(qc):
    bit_list = [] 
    phase_list = [] 
    c_bits = []
    c_phases = []

    for i in range(len(qc.data)):
        info = qc.data[i]
        gate_name = info[0].name
        if gate_name == 'h':
            phase_list.append(i)
            c_phases.append(i)
        else:
            bit_list.append(i)
            c_bits.append(i)

    li = bit_list + phase_list
    for i in range(qc.num_qubits):
        if i not in li:
            bit_list.append(i)
            c_bits.append(i)

    #making bit dictionary
    bits = {}
    for bit in bit_list:
        bits[bit * 3] = [(bit * 3 + 1), (bit * 3 + 2)]

    #making phases dictionary
    phases = {}
    for phase in phase_list:
        phases[phase * 3] = [(phase * 3 + 1), (phase * 3 + 2)]
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------


    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #THIS METHOD IS USED TO COMBINE our new RQC and QC together. This is accomplished by using the COMPOSE method.
    comp = []
    for i in range(qc.num_qubits):
        comp.append(i * 3)
    q = (qc.num_qubits * 3)
    c = q//3
    rqc = QuantumCircuit(q,c)
    rqc = rqc.compose(qc, comp)
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------

    #FIRST STEP-- RQC FOR BITS
    for i in bits.keys():
        helperQubits = bits[i]
        for qubit in helperQubits:
            rqc.cx(i,qubit)

    #FIRST STEP-- RQC For Phase
    for i in phases.keys():
        helperQubits = phases[i]
        #Consider combining these for-loops
        for qubit in helperQubits:
            rqc.cx(i,qubit)
        for qubit in helperQubits:
            rqc.h(qubit)

    rqc.barrier()
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #STEP 2 --- THIS WILL CREATE THE NOISE CHANNEL-- For BITS
    for i in bits.keys():
        error = random.randint(0,1)
        if error == 1:
            print("Bit noise at" , i // 3)
            rqc.x(i)
    #THIS WILL CREATE THE NOISE CHANNEL-- For PHASE
    for i in phases.keys():
        error = random.randint(0,1)
        if error == 1:
            print("Phase noise at", i//3)
            rqc.z(i)
    rqc.barrier()
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------

    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #STEP 3 -- This is ERROR correction
    #This is for Phases
    for i in phases.keys():
        helperQubits = phases[i]
        rqc.h(i)
        rqc.h(helperQubits)
        for qubit in helperQubits:
            rqc.cx(i,qubit)
        
    #This is for bits
    for i in bits.keys():
        helperQubits = bits[i]
        for qubit in helperQubits:
            rqc.cx(i,qubit)
            
    rqc.barrier()
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #THIS IS THE ERROR CORRECTION 
    for i in bits.keys():
        rqc.ccx(bits[i][0],bits[i][1], i)
    for i in phases.keys():
        rqc.ccx(phases[i][0], phases[i][1], i)
    rqc.barrier()

    rqc.measure(bits.keys(), c_bits)
    rqc.measure(phases.keys(), c_phases)
    return rqc

In [15]:
qc = buildQuantumCircuits.quantum_voulme(None,4)
print(qc)

TypeError: buildQuantumCircuits.quantum_voulme() missing 1 required positional argument: 'num_qubits'